# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df.drop(columns=['Unnamed: 0'], inplace=True)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kavaratti,10.57,72.64,81.79,82,100,16.11,IN,1595535366
1,Hobart,-42.88,147.33,39.99,80,77,4.70,AU,1595535395
2,Rikitea,-23.12,-134.97,66.90,63,0,16.89,PF,1595535396
3,Pakokku,21.33,95.10,81.79,72,100,10.71,MM,1595535396
4,Saint Paul Harbor,57.79,-152.41,59.00,62,40,3.36,US,1595535396


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [5]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Weight
humidity = cities_df['Humidity']

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
cleaned_df = cities_df.dropna(how='any')

In [7]:
ideal_temp = cleaned_df.loc[(cleaned_df['Max Temp']>65)&(cleaned_df['Max Temp']<75)]

In [8]:
hotel_df = ideal_temp.loc[(ideal_temp['Humidity']<50)&(ideal_temp['Cloudiness']<60)&(ideal_temp['Wind Speed']<5)]
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,Bilibino,68.05,166.44,70.88,41,0,4.99,RU,1595535399
85,Neustadt an der Waldnaab,49.71,12.09,71.60,40,1,4.83,DE,1595535125
383,Oruzgan,32.83,66.00,65.75,24,0,1.34,AF,1595535482
500,Plettenberg Bay,-34.05,23.37,65.43,41,0,3.87,ZA,1595535506


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df['Hotel Name'] = ''

C:\Users\nabis\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "type": "hotel",
    "radius": 5000,
    "key": g_key}
for index, row in hotel_df.iterrows():
    
    # get the lat and long for each city and form location parameter
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # assemble query url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print('Sorry could not find a hotel within parameters entered.')
        
hotel_df.head()

C:\Users\nabis\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
16,Bilibino,68.05,166.44,70.88,41,0,4.99,RU,1595535399,Bilibino
85,Neustadt an der Waldnaab,49.71,12.09,71.60,40,1,4.83,DE,1595535125,Weiden
383,Oruzgan,32.83,66.00,65.75,24,0,1.34,AF,1595535482,Chora
500,Plettenberg Bay,-34.05,23.37,65.43,41,0,3.87,ZA,1595535506,Plettenberg Bay


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Add layer
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))